# Init 

In [1]:
import config as c 
from src.data import CombiDataset,make_loader,downsampling
from src.model import Model 
from src.freia_funcs import * 
from src.utils import * 
from tqdm import tqdm 
import tifffile  
import os 
import pandas as pd 
import torch.nn.functional as F 

dataset_dir = c.dataset_dir
class_name = 'bottle'
mode = 'feature'
train = 'train'

my_experiment


/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Train Teacher 

In [ ]:
from train_teacher import train as train_teacher 
all_classes = [d for d in os.listdir(c.dataset_dir) if os.path.isdir(os.path.join(c.dataset_dir, d))]
all_classes.remove('.ipynb_checkpoints')
all_classes.remove('split_csv')
for class_name in all_classes:
    print(f'\n Class : {class_name}')
    save_dir = os.path.join('./saved_models',dataset_dir.split('/')[-1],class_name)
    
    trainset = CombiDataset(dataset_dir,class_name,mode,train,device=c.device)
    trainloader = make_loader(trainset,shuffle=True)
    
    teacher = Model().to(c.device)
    train_teacher(teacher,trainloader,save_dir)

# Train Student 

In [ ]:
from train_student import train as train_student
all_classes = [d for d in os.listdir(c.dataset_dir) if os.path.isdir(os.path.join(c.dataset_dir, d))]
all_classes.remove('.ipynb_checkpoints')
all_classes.remove('split_csv')
for class_name in all_classes:
    print(f'\t Class : {class_name}')
    save_dir = os.path.join('./saved_models',dataset_dir.split('/')[-1],class_name) 
    teacher = torch.load(f'./saved_models/MVtecAD/{class_name}/teacher_best.pt').to(c.device)
    student = Model(nf=not c.asymmetric_student, channels_hidden=c.channels_hidden_student, n_blocks=c.n_st_blocks).to(c.device)
    train_student(teacher,student,trainloader,save_dir)

# Eval 

In [ ]:
from eval import eval 
score = {} 
all_classes = [d for d in os.listdir(c.dataset_dir) if os.path.isdir(os.path.join(c.dataset_dir, d))]
all_classes.remove('.ipynb_checkpoints')
all_classes.remove('split_csv')

for class_name in all_classes:
    print(f'\n Class : {class_name}')
    save_dir = os.path.join('./saved_models',dataset_dir.split('/')[-1],class_name)


    testset = CombiDataset(dataset_dir,class_name,mode,'test',device=c.device)
    testloader = make_loader(testset)

    teacher = torch.load(f'./saved_models/MVtecAD/{class_name}/teacher_best.pt')
    student = torch.load(f'./saved_models/MVtecAD/{class_name}/student_best.pt')

    mean_img_auc,max_img_auc = eval(teacher,student,testloader)
    score[class_name] = {'max' : max_img_auc,'mean' : mean_img_auc}
    
    